In [ ]:
# !pip install anthropic

# Anthropic's Tool Use (Function Calling) with Claude 3

## Briefing
This code demonstrates the usage of Anthropic's Tool use (function calling) feature with the Claude 3 model. The purpose of this exercise is to explore how Claude 3 can interact with internal/external tools and retrieve relevant information based on user queries.

## Prerequisites
To run this code, the user should obtain the following API keys:
1. Anthropic Claude 3 API Key: You can get this key from [anthropic.com/api](https://www.anthropic.com/api).
2. Ninja API Key: You can get a free API key from [api-ninjas.com](https://api-ninjas.com/).

Make sure to set these API keys as environment variables before running the code.

## Code Overview
The code defines several functions:
- `send_anthropic_request`: Sends a request to the Anthropic API with the provided model, messages, tools, and max_tokens.
- `process_tool_call`: Processes the tool call based on the provided tool_name and tool_input.
- `chat_with_claude`: Engages in a conversation with Claude 3 based on the provided user_message, handling tool usage and processing the response.
- `city_info`: Provides information about a given city, including its name, population, location coordinates, and capital status.
- `external_api_country_ninja`: Retrieves country information from an external API [api-ninjas.com](https://api-ninjas.com/).
- `country_info`: Provides information about a given country, including its currency, population, refugee count, and homicide rate.

The code sets up the necessary environment variables, initializes the Anthropic client, and defines the tools that Claude 3 can use.

## Example Queries
At the end of the code, two example queries are made to Claude 3:
1. "do you know anything about United Kingdom"
2. "do you know anything about London"

Claude 3 processes these queries, uses the appropriate tools (`country_info` and `city_info`) to retrieve relevant information, and provides the final answers.

The final answers are printed with colored formatting for better readability.

In [ ]:
import anthropic
import requests
import os
from dotenv import load_dotenv

load_dotenv()  # Load the environment variables from .env file

api_key = os.environ.get("ANTHROPIC_API_KEY")
if api_key is None:
    raise ValueError("The 'ANTHROPIC_API_KEY' environment variable is not set.")

ninja_api_key = os.environ.get("NINJA_API_KEY")
if ninja_api_key is None:
    raise ValueError("The 'NINJA_API_KEY' environment variable is not set.")

client = anthropic.Anthropic(api_key=api_key)



def city_info(city):
    """
    Provides information about a given city, including its name, population, location coordinates, and capital status.
    
    Note: The city information in this function is hardcoded for demonstration purposes only.
    In a real-world scenario, you would replace this with actual data retrieval from a reliable source.
    
    
    Args:
        city (str): The name of the city.
        
    Returns:
        str: A string representation of a dictionary containing the city information.
        
    Raises:
        SyntaxError, ZeroDivisionError, NameError, TypeError, OverflowError: If an invalid city is provided.
    """
    if city:
        try:
            return str({
                "city_name": city,
                "population": "8.9M",
                "location-X": 111111,
                "location-Y": 222222,
                "capital": True
            })
        except (SyntaxError, ZeroDivisionError, NameError, TypeError, OverflowError):
            return "Error: Invalid city"
    else:
        return "There is no City info given"

def external_api_country_ninja(country):
    """
    Retrieves country information from an external API (api-ninjas.com).
    
    Args:
        country (str): The name of the country.
        
    Returns:
        dict: The JSON response from the API containing the country information.
        str: An error message if the API request fails.
    """
    api_url = f"https://api.api-ninjas.com/v1/country?name={country}"
    response = requests.get(api_url, headers={'X-Api-Key': ninja_api_key})
    if response.status_code == requests.codes.ok:
        response = response.json()
        return response
    else:
        return "Error:", response.status_code, response.text

def country_info(country):
    """
    Provides information about a given country, including its currency, population, refugee count, and homicide rate.
    
    Args:
        country (str): The name of the country.
        
    Returns:
        str: A string representation of a dictionary containing the country information.
        
    Raises:
        SyntaxError, ZeroDivisionError, NameError, TypeError, OverflowError: If an invalid country is provided.
    """
    if country:
        try:
            result = external_api_country_ninja(country)
            if result:
                return str({
                    "currency": result[0]['currency']['code'],
                    "population": result[0]['population'],
                    "refugees": result[0]['refugees'],
                    "homicide_rate": result[0]['homicide_rate']
                })
            else:
                return "The external api for country did not work"
        except (SyntaxError, ZeroDivisionError, NameError, TypeError, OverflowError):
            return "Error: Invalid country"
    else:
        return "There is no Country info given"

In [ ]:
tools = [
    {
        "name": "info_about_city",
        "description": "gives info about the city, its population and location in X and Y axis",
        "input_schema": {
            "type": "object",
            "properties": {
                "city": {
                    "type": "string",
                    "description": "The name of the city, For instance; Milan, Beijing etc."
                }
            },
            "required": ["city"]
        }
    },
    {
        "name": "info_about_country",
        "description": "gives info about the country, its population, currency, refugees rate and homicide rate",
        "input_schema": {
            "type": "object",
            "properties": {
                "country": {
                    "type": "string",
                    "description": "The name of the country, For instance; Turkey, Italy, Russia etc."
                }
            },
            "required": ["country"]
        }
    }
]

In [ ]:

def send_anthropic_request(model, messages, tools, max_tokens=4096):
    """
    Sends a request to the Anthropic API with the provided model, messages, tools, and max_tokens.
    Returns the response from the API.
    """
    response = client.beta.tools.messages.create(
        model=model,
        max_tokens=max_tokens,
        tools=tools,
        messages=messages
    )
    return response

def process_tool_call(tool_name, tool_input):
    """
    Processes the tool call based on the provided tool_name and tool_input.
    Returns the result of the corresponding tool function.
    """
    if tool_name == "info_about_city":
        return city_info(tool_input["city"])
    elif tool_name == "info_about_country":
        return country_info(tool_input["country"])

def chat_with_claude(user_message):
    """
    Engages in a conversation with Claude 3 based on the provided user_message.
    Handles tool usage and processes the response accordingly.
    Returns the final response from Claude 3.
    """
    print(f"\n{'='*50}\nUser Message: {user_message}\n{'='*50}")
    
    # Sending the initial message to Claude 3
    message = client.beta.tools.messages.create(
        model="claude-3-opus-20240229",
        max_tokens=4096,
        messages=[{"role": "user", "content": user_message}],
        tools=tools,
    )
    
    print(f"\nInitial Response:")
    print(f"Stop Reason: {message.stop_reason}")
    print(f"Content: {message.content}")
    
    # Check if the response indicates a tool use
    if message.stop_reason == "tool_use":
        tool_use = next(block for block in message.content if block.type == "tool_use")
        tool_name = tool_use.name
        tool_input = tool_use.input
        
        print(f"\nTool Used: {tool_name}")
        print(f"Tool Input: {tool_input}")
        
        # Process the tool result
        tool_result = process_tool_call(tool_name, tool_input)
        print(f"Tool Result: {tool_result}")
        
        # Send a follow-up message with the tool result
        response = client.beta.tools.messages.create(
            model=MODEL_NAME,
            max_tokens=4096,
            messages=[
                {"role": "user", "content": user_message},
                {"role": "assistant", "content": message.content},
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "tool_result",
                            "tool_use_id": tool_use.id,
                            "content": tool_result,
                        }
                    ],
                },
            ],
            tools=tools,
        )
    else:
        # If no tool was used, use the initial response
        response = message
    
    # Extracting the final response text
    final_response = next(
        (block.text for block in response.content if hasattr(block, "text")),
        None,
    )
    
    print(response.content)
    print(f"\nFinal Response: {final_response}")
    
    return final_response


In [ ]:
q = chat_with_claude("do you know anything about United Kingdom")
# Final Answer title will be red color, and its content will be magenta.
print("----"*20 + "\n\n" + "\033[31mFinal Answer:\033[0m" + f" \n \033[35m{q}\033[0m \n")



In [ ]:
q = chat_with_claude("do you know anything about London")
# Final Answer title will be red color, and its content will be magenta.
print("----"*20 + "\n\n" + "\033[31mFinal Answer:\033[0m" + f" \n \033[35m{q}\033[0m \n")
